# **PROJET DATA ENGINEERING**

## ARCHITECTURE DU PROJET

In [0]:
CATALOG = "workspace"
SCHEMA  = "xhadeezeydia"
VOLUME  = "capstoneipsl"

spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")

VOLUME_ROOT = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"
PROJECT_ROOT = f"{VOLUME_ROOT}/ecommerce_project"

DIRECTORIES = [

    # BRONZE
    "data/bronze/main",
    "data/bronze/enrich",

    # SILVER
    "data/silver/main_clean",
    "data/silver/enrich_clean",
    "data/silver/joined",

    # GOLD
    "data/gold/marts",
    "data/gold/aggregates",
    "data/gold/exports",

    # CODE
    "src/ingestion",
    "src/transforms",
    "src/quality",
    "src/utils",

    # ORCHESTRATION
    "notebooks",
    "configs",

    # REPORTS
    "reports/data_quality",
    "reports/benchmarks"
]


for d in DIRECTORIES:
    path = f"{PROJECT_ROOT}/{d}"
    dbutils.fs.mkdirs(path)
    print(f"✓ Created: {path}")


dbutils.fs.ls(f"{PROJECT_ROOT}/data")

## L'Ingestion et l'Amplification(BRONZE)

In [0]:
# 1. Chargement et Fusion (Source 1)
df_raw = spark.read.csv(f"{PROJECT_ROOT}/data/bronze/main/*.csv", header=True, inferSchema=True)

# 2. Amplification Massive 
df_amplified = df_raw
for _ in range(14):
    df_amplified = df_amplified.unionAll(df_raw)

# 3. Écriture en PARQUET
bronze_main_path = f"{PROJECT_ROOT}/data/bronze/main/full_data.parquet"
df_amplified.write.mode("overwrite").parquet(bronze_main_path)

# 4. Source 2 (Enrichissement)
enrich_data = [("electronics", "High-Tech", 0.20), ("appliances", "Home", 0.15), ("computers", "IT", 0.18)]
df_enrich = spark.createDataFrame(enrich_data, ["category_code_prefix", "category_department", "margin_rate"])
df_enrich.write.mode("overwrite").parquet(f"{PROJECT_ROOT}/data/bronze/enrich/static_ref.parquet")

# 5. Validation de la taille
size_gb = sum(f.size for f in dbutils.fs.ls(bronze_main_path) if f.name.endswith(".parquet")) / (1024**3)
print(f"✅ BRONZE VALIDE : {size_gb:.2f} GB | Format: Parquet")

In [0]:
# On passe à 20 fois la base (1 initiale + 19 unions)
df_amplified = df_raw
for _ in range(19): 
    df_amplified = df_amplified.unionAll(df_raw)

# On ré-écrit par-dessus
df_amplified.write.mode("overwrite").parquet(bronze_main_path)

# On re-vérifie
size_gb = sum(f.size for f in dbutils.fs.ls(bronze_main_path) if f.name.endswith(".parquet")) / (1024**3)
print(f"🚀 NOUVELLE TAILLE BRONZE : {size_gb:.2f} GB")

In [0]:
df_amplified.count()
